# End-to-End Machine Learning Project

Projet du chapitre 2 du livre de Aurélien Géron, où l'on va passer par les étapes normales pour s'occuper d'un projet de ML :

- Regarder le projet dans son ensemble;
- Obtenir les données;
- Découvrir et visualiser les données pour avoir un aperçu;
- Préparer les données pour l'algo de ML;
- Sélectionner un modèle et l'entraîner;
- Affiner le réglage du modèle;
- Présenter sa solution;
- Lancer, gérer et maintenir le système.

Dans ce projet, on va agir comme un data scientist récemment employé par une entreprise de gestion immobilière.

### Travailler avec de vraies données

Il est souvent plus intéressant de travailler avec de vraies données existantes plutôt que des jeux de données artificiels.
Ici, on utilisera le jeu de données "California Housing Prices" du StatLab Repository, basé sur les données du California Census de 1990.

## Regarder le projet dans son ensemble

La première étape est d'utiliser le California Census pour construire un modèle des prix des maisons dans l'état. Ces données incluent des mesures comme la population, le revenu médian, et le prix médian du loyer pour chaque "bloc", qu'on appelera districts (regroupement de 600 à 3000 habitants).

Le modèle devrait apprendre à partir de ces données et devrait être capable de prédire le prix du loyer médian dans chaque district, étant donné les autres mesures.

### Cerner le problème

Il est important de se demander quel est l'objectif derrière la construction de ce modèle. Connaître l'objectif est important car il va permettre de déterminer comment on va cerner le problème, quel algorithme on va choisir, quelle mesure de performance on va utiliser pour évaluer le modèle, et quelle quantité d'effort on va fournir pour le "tordre".

Dans ce projet, la sortie de notre modèle sera donné à un autre système de ML. Ce système va déterminer s'il est intéressant d'investir dans une zone donnée ou non. Il est important que cela soit fait de manière correcte, car cela affectera directement les revenus.

Il est aussi important de savoir comment la solution est gérée avant l'implémentation du modèle de ML.
Dans notre cas, c'est fait à la main, et quand les données sont insuffisantes, les employés dédiés à la tâche ont des estimations pas top. S'ils parviennent à estimer le prix médian, il y a souvent un écart de plus de 20% avec la réalité. D'où l'idée d'entraîner un modèle pour prédire le loyer médian d'un distric, étant donné qu'on a des données pour les autres districts. Les données du census ont l'air d'être un très bon jeu de données à exploiter dans ce but, puisqu'il inclut les prix médians des loyers des centaines de districts, ainsi que d'autres données.

Avec ces informations, on peut commencer à concevoir le système. Premièrement, on va devoir cerner le problème : supervisé ou non ? ou apprentissage par renforcement ?

On a ici clairement affaire à un _problème supervisé_, puisqu'on va utiliser des jeux d'entraînement _labelisés_.

Il s'agit également d'une tâche de _régression_, puisqu'on doit _prédire une valeur_. Plus spécifiquement, d'une _régression multiple_, puisque le système va utiliser plusieurs caractéristiques pour faire une prédiction (revenu médian, population du district, etc.).

C'est également une régression _univariée_ puisqu'on essayer de prédire seulement une valeur pour chaque district. Si on essayait de prédire de multiples valeurs par discrit, il s'agirait d'un problème _multivarié_.

Enfin, puisqu'il n'y a pas de flot de données continu arrivant au système, il n'est pas nécessaire de s'ajuster à un changement rapide des données, et le jeu de donnée est assez petit pour passer en mémoire, donc le _batch learning_ devrait aller très bien pour ce cas (si les données étaient énormes, on aurait pu faire du batch learning sur plusieurs serveurs, ou utiliser du _online learning_).

### Sélectionner une mesure de performance

L'étape suivante est la sélection d'une mesure de performance. Une mesure de performance typique pour les problèmes de régression est la RMSE (Root Mean Square Error). Elle donne une idée de combien d'erreurs le système fait typiquement dans ses prédictions, avec un poids plus important pour les grosses erreurs.

Equation de la RMSE (Racine de l'erreur quadratique moyenne) :

$sqrt{\sum_{i=1}^{10} t_i}$ (à compléter)

Même si le RMSE est est généralement la mesure de preformance privilégiée pour les tâches de régression, il est possible dans certains contextes d'utiliser d'autres équations. Par exemple, supposons qu'il y ait des districts _outliers_ ; dans ce cas, on pourrait considérer l'utilisation de la _mean absolute error_ (MAE, aussi appelée _average absolute error_).

Equation de l'Erreur absolue moyenne : (à remplir)

La RMSE et la MAE sont des moyens de mesurer la distance entre deux vecteurs : le vecteur de prédiction et le vecteur de valeur cible. De nombreuses mesures de distances, ou _normes_, sont possibles :
- Calculer la RMSE correspond à la _norme euclidienne_; également appelé $l_{2}$ _norm_
- Calculer la MAE à la $l_{1}$ _norm_. Parfois appelée norme de Manhatthan, parce qu'elle mesure la distance entre deux points dans une ville ou on ne peut que se déplacer le long de blocs.
- Plus généralement, la $l_{k}$_norm_ d'un vecteur __v__  contenant _n_ éléments est défini comme ||v||_{k} = (|v_{0}|^k) ... (à compléter)
- Plus l'index de la norme est élevé, plus il va se focaliser sur les grande valeurs et négliger les petites valeurs. C'est pourquoi le RMSE est plus sensible aux _outliers_ que la MAE. Mais quand les _outliers_ sont exponentiellement rare, le RMSE a une très bonne performance et est généralement préféré.

### Vérifier les suppositions

Enfin, il est important de lister et de vérifier les suppositions qui ont été faites ;  cela permet de repérer des problèmes dès le début.

Par exemple, dans notre cas, les prix des districts que notre système va fournir en sortie seront donnés à un autre système de ML et on assume que les prix vont être utilisés en tant que tels. Mais que se passerait-il si le système en aval converti les prix en catégories (comme "bon marché", "cher") et utilise ensuite les catégories plutôt que les prix ? Dans ce type de situation, donner parfaitement le juste prix n'est pas important du tout, le système n'ayant besoin que d'une bonne catégorie. Dans ce cas, le problème ne devrait pas être traîté comme une tâche de régression mais une tâche de classification. C'est le genre de chose qu'on ne souhaite pas découvrir après avoir mis en place le modèle.

Heureusement, dans notre cas, il s'agit bien d'une régression et on va pouvoir passer à la partie code !


## Obtenir les données

Il est temps de mettres les mains dans le cambouis.

### Créer l'espace de travail

Il est recommandé de créer un espace de travail virtuel pour ne pas avoir de conflit entre les bilbiothèques d'un projet.

### Télécharger les données

Habituellement, on se retrouve à devoir communiquer avec une base de données relationnelle qui demande des autorisations.

Ici, on va juste télécharger un fichier compressé, _housing.csv_ contenant toutes les données.

On va dans un premier temps créer une fonction qui va télécharger le fichier (plutôt que d'aller le prendre avec un navigateur).

Avoir une fonction qui va chercher automatiquement les résultats est utile surtout si les données sont ammenées à changer régulièrement. C'est également utile dans le cas où on travaille sur plusieurs machines.


In [9]:
import os
import tarfile
import urllib

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path,exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close

L'appel à la fonction __fetch_housing_data()__ va créer un répertoire _dataset/housing_ dans l'espace de travail, télécharger le fichier _housing.tgz_, et en extraire le fichier _housing.csv_ dans ce répertoire.

On va maintenant charger les données en utilisant pandas. Ici aussi, il est conseillé de faire une fonction pour charger les données.

In [3]:
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

On charge et on affiche 

In [12]:
fetch_housing_data()
housing = load_housing_data()
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


Chaque ligne représente un district. Il y a 10 attributs : longitude, latitude, housing_median_age, total_rooms, total_bedrooms, population, households, median_income, median_house_value et ocean_proximity

La méthode __info()__ est utile pour obtenir une description rapide, en particulier le nombre de lignes, le type de chaque attribut, et le nombre de valeurs _nonnull_.

In [13]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


Il y a 20640 instances dans le jeu de données. C'est peu pour les standards du ML, mais cela ira pour débuter.

On remarquera que l'attribut *total_bedrooms* comptabilise 20433 valeurs non nulles, ce qui veut dire qu'il manque des données pour 207 districts. On s'occupera de ce problème plus tard.

Tous les attributs sont de type numérique, sauf *ocean_proximity* qui est de type objet. Comme il s'agit d'un csv, c'est certainement du texte. L'observation faite sur les 5 premières lignes nous permet de penser qu'il peut s'agir d'un attribut de catégorie.

Vérifions cela :

In [14]:
housing["ocean_proximity"].value_counts()

<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: ocean_proximity, dtype: int64

Regardons à présent les autres champs. La méthode **describe()"** présente un résumé des attributs numériques.

In [15]:
housing.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000
